In [1]:
import os, serial, time
from numpy import *
from colorsys import *

# Communication functions

In [2]:
def updateLed( sObj, dat=None, channel=0 ):
    if dat is None:
        nLeds = 1024
        ledDat = bytearray( os.urandom( nLeds*3 ) )
    elif type(dat) is int:
        nLeds = dat
        ledDat = bytearray( os.urandom( nLeds*3 ) )
    else:
        nLeds = len(dat)
        ledDat = bytearray( dat )
    sDat = bytes("LED {0} {1}\n".format(channel, len(ledDat)), "utf8") + ledDat
    sObj.write( sDat )

# Open serial port connection

In [6]:
try:
    s.close()
except:
    pass
s = serial.Serial( "/dev/ttyACM1", timeout=1 )

# Get ID and Software Version

In [7]:
s.read_all()          #Clear receive buffer
s.write(b"\n*IDN?\n") #First \n clears send buffer
s.read_until()

b'ID:MB:V0.1\n'

# Get Switch state

In [5]:
#%%timeit   #1000 loops, best of 3: 1.78 ms per loop
s.write(b"SW?\n")
s.read_until()

b'SW:f6ffffffffffffff0000ff0000000000000000000000000000000000000000000000000000000000\n'

# Setup high and low speed WS2811 / WS2812 LED strands

In [21]:
s.write(b"LEC 0 3200000\n")
s.write(b"LEC 1 1700000\n")
s.write(b"LEC 2 3200000\n")

14

# Benchmark LED throughput (worst case)

In [17]:
%%timeit
x = array( ones(1024*3), dtype=uint8)
updateLed(s, x, 0)
updateLed(s, x, 1)
updateLed(s, x, 2)
# [Only USB comm.] 10 loops, best of 100: 36.0 ms per loop (28 Hz)
# [USB + SPI send] 10 loops, best of 100: 45.5 ms per loop (22 Hz)

40 ms ± 998 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


It looks like the USB communication is the bottleneck at the moment. Note that the firmware can transmit on several channels simultaneously but will block if a channel is updated, which has not finished transmitting yet.

**TLDR:** If you need > 30 Hz refresh rate, do not connect more than 512 LEDs per string

# Play with LEDs

### Glitch test

Do many sets of the same values. LEDs should not flicker at all!

In [22]:
x = zeros( 59*3, dtype=uint8 )
x[0:-1][::2] = 1
#x[-6:] = 1
for i in range(8000):
    updateLed( s, x, 0 )
    updateLed( s, x, 1 )
    updateLed( s, x, 2 )

KeyboardInterrupt: 

### Turn ON one color after another

In [23]:
updateLed(s, zeros(100*3, dtype=uint8), 0 )
updateLed(s, zeros(100*3, dtype=uint8), 1 )
updateLed(s, zeros(100*3, dtype=uint8), 2 )

In [26]:
while(True):
    x = zeros( 60*3, dtype=uint8 )
    for i in range(len(x)):
        x[i] = 255
        updateLed(s, x, 0)
        updateLed(s, x, 1)
        updateLed(s, x, 2)
        time.sleep(1/60)
    updateLed(s, zeros(100*3, dtype=uint8), 0 )
    updateLed(s, zeros(100*3, dtype=uint8), 1 )
    updateLed(s, zeros(100*3, dtype=uint8), 2 )
    time.sleep(0.5)

KeyboardInterrupt: 

### LED ticker on button

In [27]:
curLed = 0
z = zeros(100*3, dtype=uint8)
updateLed(s, z, 0 )
updateLed(s, z, 1 )
updateLed(s, z, 2 )

In [28]:
if curLed > 48:
    curLed = 0
z[:] = 0
z[ curLed*3:curLed*3+3 ] = [255]*3
updateLed(s, z, 1 )
print( curLed )
curLed += 1

0


### All LEDs same color

In [27]:
x = array( [5, 25, 25]*60, dtype=uint8 )
updateLed( s, x, 0 )
updateLed( s, x, 1 )
updateLed( s, x, 2 )

### Gamma corrected fade-UP

In [30]:
# Precompute quadratic brightness values
bVals = array( arange(33)**2/4, dtype=uint8 )
bVals[-1] = 255

In [31]:
for bVal in bVals:
    x = array( [bVal, bVal, bVal]*5, dtype=uint8 )
    updateLed( s, x, 0 )
    time.sleep(0.03)

### Each LED a random color

In [23]:
# Set it once
x = array(random.randint(0,255,60*3),dtype=uint8)
#updateLed(s, x, 0)
updateLed(s, x, 1)
#updateLed(s, x, 2)

In [28]:
# Set it in a loop
while True:
    x = array(random.randint(0,2,58*3)*25,dtype=uint8)
    updateLed( s, x, 0 )
    updateLed( s, x, 1 )
    updateLed( s, x, 2 )
    time.sleep(0.05)

KeyboardInterrupt: 

# Rainbow

### Darken LEDs

In [29]:
updateLed( s , zeros(70*3, dtype=uint8), 0 )
updateLed( s , zeros(70*3, dtype=uint8), 1 )
updateLed( s , zeros(70*3, dtype=uint8), 2 )

### Setup color values

In [30]:
nCols = 80
x = arange(nCols)/(nCols-1)
x2 = list(map( hsv_to_rgb, x, ones(nCols), ones(nCols)*1.0))
rawOut = array( array(x2) * 255, dtype=uint8 )

### Roll color values through the string

In [31]:
#z = zeros(3*48, dtype=uint8)
while(True):
    #updateLed( s, array(rawOut[:60]/5, dtype=uint8), 0 )
#    z[24*3:24*3+3] = rawOut[0]
    updateLed( s, rawOut[:49], 1 )
    #updateLed( s, rawOut[:60], 2 )
    rawOut = roll(rawOut, 3)
    time.sleep(1/30)

KeyboardInterrupt: 

# Enable reporting of Switch Events

In [17]:
#SWE   : <OnOff> En./Dis. reporting of switch events.
s.write(b"SWE 1\n")

6

# Play with Relay Outputs

### Clean power relay contacts

In [12]:
while True:
    s.write(b"SOE 1\n")
    time.sleep(0.1)
    s.write(b"SOE 0\n")
    time.sleep(0.1)

KeyboardInterrupt: 

In [1]:
try:
    s.close()
except:
    pass

In [2]:
s = serial.Serial( "/dev/ttyACM0", timeout=1 )

NameError: name 'serial' is not defined

### Flush serial input buffer

In [8]:
print( s.read_all() )
s.write(b"\n*IDN?\n")
print( s.read_until() )

b''
b'ID:MB:V0.1\n'


### Enable Power Relay

In [9]:
s.write(b"SOE 1\n")

6

### Set a Solenoid to a specific power

In [53]:
#OUT   : <hwIndex> <PWMlow>
s.write(b"OUT 0x40 0\n")

11

### Trigger a solenoid pulse

In [72]:
#OUT   : <hwIndex> <PWMlow> [tPulse] [PWMhigh]
s.write(b"OUT 0x40 0 20 2\n")
s.write(b"OUT 0x41 0 20 2\n")
s.write(b"OUT 0x42 0 20 2\n")

16

### Mushrooms

In [20]:
s.write( "OUT 0x40 0 500 6\n" )

TypeError: unicode strings are not supported, please encode to bytes: 'OUT 0x40 0 500 6\n'

In [60]:
for i in (0x40, 0x41, 0x42):
    s.write( "OUT {0} 0 50 2\n".format(i).encode("utf8") )
    time.sleep( 0.1 )
s.write( b"OUT 0x3D 0 50 1000\n" )

19

### Flippers

In [44]:
for i in (0x3C, 0x3E):
    s.write( "OUT {0} 0 50 2000\n".format(i).encode("utf8") )
    time.sleep( 0.1 )

# Setup quick-fire rules

Trigger Second Solenoid when Second Switch closes

In [52]:
#RUL <ID> <IDin> <IDout> <trHoldOff> <tPulse> <pwmOn> <pwmOff> <bPosEdge>
s.write(b"RUL 0 0x0000 64 30 20 5 0 1\n")

28

First switch --> First Solenoid. Add a 1000 ms hold-off time (triggers max. once a second). 

In [53]:
s.write(b"RUL 1 0x0000 64 1000 100 5 0 1\n")

31

### Disable the rules

In [75]:
for i in range(16):
    s.write("RULE {0} 0\n".format(i).encode("UTF8"))

### Rules for basic Flipper operation (Fan-Tas-Tic)

In [10]:
#RUL    <ID><IDin><IDout><trHoldOff><tPulse><pwmOn><pwmOff><bPosEdge>
# Note that buttons are active low
# Flipper rules
# Attack + Hold on neg. edge
s.write(b"RUL 0 0x4C 0x3C 200 75 1500 500 0\n")
s.write(b"RUL 1 0x4F 0x3E 200 75 1500 500 0\n")
# Release on pos. edge
s.write(b"RUL 2 0x4C 0x3C 0 0 0 0 1\n")
s.write(b"RUL 3 0x4F 0x3E 0 0 0 0 1\n")
# Jet bumper rules
rulId = 4
for hwIndexIn, hwIndexOut in zip( (0x05, 0x04, 0x03, 0x1A, 0x28, 0x17), (0x40, 0x41, 0x42, 0x3D, 0x44, 0x3F) ):
    if hwIndexOut in (0x3D, 0x3F):
        power = 2500
    else:
        power = 4
    rulStr = "RUL {0} {1} {2} 0 20 {3} 0 0\n".format( rulId, hwIndexIn, hwIndexOut, power )
    print( rulStr )
    s.write( rulStr.encode("UTF8") )
    rulId += 1
    # disable debouncing for the jet bumper inputs
    #s.write( "DEB {0} 0\n".format(hwIndexIn).encode("UTF8") )
# Captive ball rules
for hwIndexIn, hwIndexOut in zip( (0x16, 0x23, 0x00), (0x46, 0x47, 0x43) ):
    rulStr = "RUL {0} {1} {2} 1000 75 3 0 1\n".format( rulId, hwIndexIn, hwIndexOut )
    print( rulStr )
    s.write( rulStr.encode("UTF8") )
    rulId += 1

    

RUL 4 5 64 0 20 4 0 0

RUL 5 4 65 0 20 4 0 0

RUL 6 3 66 0 20 4 0 0

RUL 7 26 61 0 20 2500 0 0

RUL 8 40 68 0 20 4 0 0

RUL 9 23 63 0 20 2500 0 0

RUL 10 22 70 1000 75 3 0 1

RUL 11 35 71 1000 75 3 0 1

RUL 12 0 67 1000 75 3 0 1



    #Flippers
    RUL 0 0x4C 0x3C 200 75 3000 500 0
    RUL 1 0x4F 0x3E 200 75 3000 500 0
    RUL 2 0x4C 0x3C 0 0 0 0 1
    RUL 3 0x4F 0x3E 0 0 0 0 1

    #Jet bumpers
    RUL 4 5 64 0 15 4 0 1
    RUL 5 4 65 0 15 4 0 1
    RUL 6 3 66 0 15 4 0 1
    RUL 7 26 61 0 15 4000 0 1
    
    #Captive holes
    #R
    RUL 8 0x16 0x46 500 75 3 0 1
    #L
    RUL 9 0x23 0x47 500 75 3 0 1
    #T
    RUL 10 0x00 0x43 500 75 3 0 1
    

In [33]:
s.write(b"OUT 0x48 0 200 2\n")

17

In [19]:
s.close()


In [18]:
s.write(b"SWE 0\n")

6

# Pseudo console

In [11]:
while True:
    rx = s.read_until()
    if len( rx ) > 0:
        print( rx )

b'SE:04f=0 \n'
b'SE:018=1 030=1 \n'
b'SE:04f=1 \n'
b'SE:018=0 030=0 \n'
b'SE:030=1 033=1 \n'
b'SE:020=0 \n'
b'SE:020=1 \n'
b'SE:04f=0 \n'
b'SE:018=1 \n'
b'SE:04f=1 \n'
b'SE:04c=0 \n'
b'SE:018=0 \n'
b'SE:04d=0 \n'
b'SE:01b=1 \n'
b'SE:04d=1 \n'
b'SE:04c=1 \n'
b'SE:01b=0 \n'
b'SE:004=0 \n'
b'SE:004=1 \n'
b'SE:00a=0 \n'
b'SE:00a=1 \n'
b'SE:005=0 \n'
b'SE:005=1 \n'
b'SE:01c=0 \n'
b'SE:01c=1 \n'
b'SE:019=0 \n'
b'SE:019=1 \n'
b'SE:01c=0 \n'
b'SE:01c=1 \n'
b'SE:01c=0 \n'
b'SE:01c=1 \n'
b'SE:04c=0 \n'
b'SE:04d=0 \n'
b'SE:01b=1 \n'
b'SE:04d=1 \n'
b'SE:04c=1 \n'
b'SE:01b=0 \n'
b'SE:00c=0 \n'
b'SE:00d=0 \n'
b'SE:00d=1 \n'
b'SE:00c=1 \n'
b'SE:04f=0 \n'
b'SE:018=1 \n'
b'SE:04f=1 \n'
b'SE:018=0 \n'
b'SE:01a=0 \n'
b'SE:01a=1 \n'
b'SE:01a=0 \n'
b'SE:01a=1 \n'
b'SE:030=0 033=0 \n'
b'SE:030=1 033=1 \n'
b'SE:030=0 033=0 \n'
b'SE:030=1 033=1 \n'
b'SE:020=0 \n'
b'SE:020=1 \n'
b'SE:003=0 \n'
b'SE:003=1 \n'
b'SE:004=0 \n'
b'SE:004=1 \n'
b'SE:004=0 \n'
b'SE:004=1 \n'
b'SE:005=0 \n'
b'SE:005=1 \n'
b'SE:013=0 \n

KeyboardInterrupt: 

In [8]:
s.close()

# MPF number grouping hack

In [86]:
def group_digits(text, separator=",", group_size=3):
    digit_list = list(text.split('.')[0])
    for i in range(len(digit_list))[::-group_size][1:]:
        digit_list.insert(i + 1, separator)
    return ''.join(digit_list)

def nGroup( text ):
    # find the numbers in the string
    number_list = [s for s in text.split() if s.isdigit()]
    # group the numbers and replace them in the string
    for item in number_list:
        grouped_item = group_digits(item)
        text = text.replace(str(item), grouped_item)
    return text

def nSci( text ):
    # find the numbers in the string
    number_list = [s for s in text.split() if s.isdigit()]
    # group the numbers and replace them in the string
    for item in number_list:
        print(item)
        grouped_item = item
        text = text.replace(str(item), grouped_item)
    return text

def getSci( val ):
    i = int(val)
    exp = 0
    while( i ):
        i //= 10
        exp += 1
    lInd = max(0,(exp-1))
    pFix = ["", "k", "M", "G", "T"]
    print( val//10**(lInd), pFix[lInd//3] )

In [25]:
nGroup("bla 124000056 dsdsds")

'bla 124,000,056 dsdsds'

In [102]:
getSci(10)

1 
